### TO DO ###

- [x] Fix no data for first week
- [x] Add cumulative to standings
- [x] Check by week when new data comes
- [ ] Add value for players who don't play
- [ ] Goal difference statistic
- [ ] Above or below average score
- [ ] Pandas Stylesheet
- [ ] Graphs

In [138]:
import requests
import pandas as pd
import numpy as np

#### Definitions ####

In [139]:
owners = ['Luke','Emma','Fat','Olivia','Claire','Matt','George']
owner_ids = ['18900','18935','18967','19180','19424','20043','20626']
team_details_url = {}
lineup_url = {}

for owner_id, owner in enumerate(owners):
    team_details_url[owner] = 'https://draft.premierleague.com/api/entry/'+owner_ids[owner_id]+'/history'
    for gameweek in range(1,39):
        lineup_url[owner,gameweek] = 'https://draft.premierleague.com/api/entry/'+owner_ids[owner_id]+'/event/'+str(gameweek)

bootstrap_url = 'https://draft.premierleague.com/api/bootstrap-static'
fantasy_bootstrap_url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
player_owner_url = 'https://draft.premierleague.com/api/league/5857/element-status'
league_details_url = 'https://draft.premierleague.com/api/league/5857/details'

current_gameweek = requests.get(bootstrap_url).json()['events']['current']
total_fantasy_players = requests.get(fantasy_bootstrap_url).json()['total_players']

In [140]:
def url_to_df(url, datatype):
    request = requests.get(url)
    json = request.json()
    df = pd.DataFrame(json[datatype])
    return df

#### Create Dataframes ####

In [141]:
player_types_df = url_to_df(bootstrap_url,'element_types')

teams_df = url_to_df(bootstrap_url,'teams')

player_owner_df = url_to_df(player_owner_url,'element_status')
player_owner_df = player_owner_df.sort_values('element', ignore_index=True)

league_entries_df = url_to_df(league_details_url,'league_entries')
league_standings_df = url_to_df(league_details_url,'standings')
league_standings_df = league_standings_df.sort_values('league_entry', ignore_index=True)
league_standings_df = pd.concat([league_entries_df[['entry_id','id','entry_name','player_first_name','waiver_pick']], league_standings_df[['last_rank','rank','event_total','total']]], axis=1)

player_info_df = url_to_df(bootstrap_url,'elements')
player_info_df['team'] = player_info_df.team.map(teams_df.set_index('id').name)
player_info_df['element_type'] = player_info_df.element_type.map(player_types_df.set_index('id').singular_name)
player_info_df.insert(3,'owner',player_owner_df.owner.map(league_entries_df.set_index('entry_id').player_first_name))

#### Functions for making specific DataFrames ####

In [150]:
def get_lineup_info(owner, gameweek = current_gameweek, with_totals = True):
    # check valid gameweek:
    if gameweek > current_gameweek:
        print("Gameweek not yet played")
        return('ERROR')
    
    # create dataframe:
    info_columns = ['name','team','position']
    points_columns = ['minutes','goals_scored','goals_conceded','penalties_missed','penalties_saved','assists','saves',
                      'own_goals','yellow_cards','red_cards','total_points']
    lineup_df = pd.DataFrame(columns=info_columns + ['value', 'selected %'] + points_columns,
                             index = url_to_df(lineup_url[owner, gameweek],'picks')['element'])
    lineup_df.insert(15,'foul points',pd.Series([], dtype=object))
    
    # all player data:
    lineup_df['name'] = lineup_df.index.map(player_info_df.set_index('id').web_name)
    lineup_df['team'] = lineup_df.index.map(player_info_df.set_index('id').team)
    lineup_df['position'] = lineup_df.index.map(player_info_df.set_index('id').element_type)
    
    # player by player data:
    for i, player in enumerate(lineup_df.index):
        for data in points_columns + ['value', 'selected %']:
            lineup_df.loc[lineup_df.index == player, data] = 0
        player_url = 'https://draft.premierleague.com/api/element-summary/'+str(player)
        player_df = url_to_df(player_url,'history')
        fantasy_player_url = 'https://fantasy.premierleague.com/api/element-summary/'+str(player)+'/'
        fantasy_player_df = url_to_df(fantasy_player_url,'history')
        if not player_df.loc[player_df['event'] == gameweek].empty:
            lineup_df.loc[lineup_df.index == player, 'value'] = fantasy_player_df.loc[fantasy_player_df['round'] == gameweek]['value'].iloc[0]/10
            lineup_df.loc[lineup_df.index == player, 'selected %'] = fantasy_player_df.loc[fantasy_player_df['round'] == gameweek]['selected'].iloc[0]/total_fantasy_players*100
            for data in points_columns:
                lineup_df.loc[lineup_df.index == player, data] = player_df.loc[player_df['event'] == gameweek][data].iloc[0]
            if not lineup_df.loc[lineup_df.index == player, 'red_cards'].iloc[0] == 0:
                lineup_df.loc[lineup_df.index == player, 'foul points'] = 3
            elif not lineup_df.loc[lineup_df.index == player, 'yellow_cards'].iloc[0] == 0:
                lineup_df.loc[lineup_df.index == player, 'foul points'] = 1
            else:
                lineup_df.loc[lineup_df.index == player, 'foul points'] = 0
    
    if with_totals:
        # add totals row:
        totals_df = pd.DataFrame(index = ['totals (inc. bench)','totals'])
        for data in info_columns:
            totals_df.loc[:, data] = ''
        for data in ['value', 'foul points'] + points_columns:
            totals_df.loc['totals (inc. bench)', data] = lineup_df[data].sum()
            totals_df.loc['totals', data] = lineup_df.head(11)[data].sum()
        totals_df.loc['totals (inc. bench)', 'name'] = sum(lineup_df['minutes']>0)
        totals_df.loc['totals', 'name'] = sum(lineup_df.head(11)['minutes']>0)
        totals_df.loc['totals (inc. bench)', 'selected %'] = lineup_df['selected %'].mean()
        totals_df.loc['totals', 'selected %'] = lineup_df.head(11)['selected %'].mean()
        totals_df.loc['totals (inc. bench)', 'team'] = lineup_df['team'].nunique()
        totals_df.loc['totals', 'team'] = lineup_df.head(11)['team'].nunique()
        lineup_df = pd.concat([lineup_df, totals_df])
        
    # make reader friendly:
    lineup_df.index.name = None
    lineup_df = lineup_df.rename(columns={'goals_scored': 'goals scored', 'goals_conceded': 'goals conceded',
                                          'own_goals': 'own goals', 'yellow_cards': 'yellow cards',
                                          'red_cards': 'red cards', 'total_points': 'total points',
                                          'penalties_missed': 'penalties missed','penalties_saved': 'penalties saved'})
    return(lineup_df)

In [153]:
get_lineup_info('Emma',2,True)

,name,team,position,value,selected %,minutes,goals scored,goals conceded,penalties missed,penalties saved,assists,saves,own goals,yellow cards,red cards,foul points,total points
252,Alisson,Liverpool,Goalkeeper,6,10.9539,90,0,0,0,1,0,3,0,0,0,0,14
62,Dunk,Brighton,Defender,5,7.59386,90,0,0,0,0,0,0,0,0,0,0,7
458,Boly,Wolves,Defender,5.5,2.28978,90,0,3,0,0,0,0,0,0,0,0,1
323,Fernández,Newcastle,Defender,5,1.53604,90,0,3,0,0,0,0,0,0,0,0,1
254,Salah,Liverpool,Midfielder,12,35.5355,90,0,0,0,0,0,0,0,0,0,0,3
241,Milner,Liverpool,Midfielder,5.5,0.192625,26,0,0,0,0,0,0,0,0,0,0,1
113,Kanté,Chelsea,Midfielder,5,4.32305,90,0,2,0,0,0,0,0,0,0,0,2
193,Hernández,Leeds,Midfielder,6,1.67581,0,0,0,0,0,0,0,0,0,0,0,0
388,Kane,Spurs,Forward,10.5,12.3874,83,1,1,0,0,4,0,0,0,0,0,21
268,Agüero,Man City,Forward,10.5,3.51586,0,0,0,0,0,0,0,0,0,0,0,0


In [154]:
def get_league_standings(Cumulative=True):
    gameweek_range = range(1,current_gameweek+1)
    # create dataframe:
    league_standings_df = pd.DataFrame(columns = gameweek_range, index = owners)
    league_standings_df.insert(current_gameweek, 'total points', 0)
    team_info_df = {}
    for owner in owners:
        team_info_df[owner] = url_to_df(team_details_url[owner],'history')
        for week in gameweek_range:
            points = team_info_df[owner].loc[team_info_df[owner]['event'] == week,'points'].iloc[0]
            if week == 1 or not Cumulative:
                league_standings_df.loc[owner, week] = points
            else:
                league_standings_df.loc[owner, week] = league_standings_df.loc[owner, week-1] + points
        if Cumulative:
            league_standings_df.loc[owner, 'total points'] = league_standings_df.loc[owner, current_gameweek]
        else:
            league_standings_df.loc[owner, 'total points'] = league_standings_df.loc[owner, :].sum()
    
    league_standings_df = league_standings_df.sort_values('total points', ascending=False)    
    return(league_standings_df)

In [155]:
get_league_standings(True)

,1,2,total points
Emma,43,105,105
Claire,39,85,85
Luke,27,84,84
Olivia,31,82,82
Matt,22,78,78
Fat,48,73,73
George,26,64,64


#### Get all info (only run when new info to retrieve) ####

In [156]:
def get_all_lineups():
    all_lineups_df = {}
    for gameweek in range(1, current_gameweek+1):
        for owner in owners:
            all_lineups_df[owner, gameweek] = get_lineup_info(owner, gameweek, True)
    return(all_lineups_df)

In [157]:
all_lineups_df = get_all_lineups()

In [159]:
all_lineups_df['Emma',2]

,name,team,position,value,selected %,minutes,goals scored,goals conceded,penalties missed,penalties saved,assists,saves,own goals,yellow cards,red cards,foul points,total points
252,Alisson,Liverpool,Goalkeeper,6,10.9539,90,0,0,0,1,0,3,0,0,0,0,14
62,Dunk,Brighton,Defender,5,7.59386,90,0,0,0,0,0,0,0,0,0,0,7
458,Boly,Wolves,Defender,5.5,2.28978,90,0,3,0,0,0,0,0,0,0,0,1
323,Fernández,Newcastle,Defender,5,1.53604,90,0,3,0,0,0,0,0,0,0,0,1
254,Salah,Liverpool,Midfielder,12,35.5355,90,0,0,0,0,0,0,0,0,0,0,3
241,Milner,Liverpool,Midfielder,5.5,0.192625,26,0,0,0,0,0,0,0,0,0,0,1
113,Kanté,Chelsea,Midfielder,5,4.32305,90,0,2,0,0,0,0,0,0,0,0,2
193,Hernández,Leeds,Midfielder,6,1.67581,0,0,0,0,0,0,0,0,0,0,0,0
388,Kane,Spurs,Forward,10.5,12.3874,83,1,1,0,0,4,0,0,0,0,0,21
268,Agüero,Man City,Forward,10.5,3.51586,0,0,0,0,0,0,0,0,0,0,0,0


#### Parse All Lineups DF ####

In [232]:
def get_league_stats_df_by_gameweek(all_lineups_df, include_bench = False):
    if include_bench:
        value_to_get = 'totals (inc. bench)'
    else:
        value_to_get = 'totals'
    stats = ['name','team','value','selected %','minutes','goals scored','goals conceded','penalties missed','penalties saved','assists','saves','own goals','yellow cards','red cards','foul points','total points']
    league_stats_df = {}
    gameweek_range = range(1,current_gameweek+1)
    for gameweek in gameweek_range:
        league_stats_df[gameweek] = pd.DataFrame(columns = stats + ['weekly position'], index = owners)
        league_stats_df[gameweek].insert(5, 'productivity', 0)
        league_stats_df[gameweek].insert(8,'goal difference',pd.Series([], dtype=object))
    
        for owner in owners:
            for stat in stats:
                league_stats_df[gameweek].loc[owner, stat] = all_lineups_df[owner, gameweek].loc[value_to_get,stat]
        league_stats_df[gameweek] = league_stats_df[gameweek].sort_values('total points', ascending=False)
        for i, owner in enumerate(league_stats_df[gameweek].index):
            league_stats_df[gameweek].loc[owner, 'productivity'] = league_stats_df[gameweek].loc[owner, 'total points']/league_stats_df[gameweek].loc[owner, 'minutes']
            league_stats_df[gameweek].loc[owner, 'goal difference'] = league_stats_df[gameweek].loc[owner, 'goals scored'] - league_stats_df[gameweek].loc[owner, 'goals conceded']
            league_stats_df[gameweek].loc[owner, 'weekly position'] = (i+1)
        league_stats_df[gameweek] = league_stats_df[gameweek].rename(columns={'name': 'active players'})
    return(league_stats_df)

In [233]:
league_stats_df_by_gameweek = get_league_stats_df_by_gameweek(all_lineups_df)
league_stats_df_by_gameweek_with_bench = get_league_stats_df_by_gameweek(all_lineups_df, True)

In [234]:
league_stats_df_by_gameweek[2][['goals scored','goals conceded','goal difference','weekly position']].sort_values('goal difference', ascending=False)

,goals scored,goals conceded,goal difference,weekly position
George,4,12,-8,6
Emma,1,11,-10,1
Claire,5,19,-14,5
Luke,4,19,-15,2
Matt,5,20,-15,3
Olivia,4,20,-16,4
Fat,1,20,-19,7


In [236]:
def get_league_stats_df_by_stat(league_stats_df_by_gameweek, stat, total_type = 'none', sort_by_week = 0):
    gameweek_range = range(1,current_gameweek+1)
    
    league_stats_df_by_stat = pd.DataFrame(columns = gameweek_range, index = owners)
    if total_type == 'total':
        league_stats_df_by_stat.insert(current_gameweek,'totals',pd.Series([], dtype=object))
    
    for owner in owners:
        for gameweek in gameweek_range:
            league_stats_df_by_stat.loc[owner, gameweek] = league_stats_df_by_gameweek[gameweek].loc[owner, stat]
        if total_type == 'mean' or total_type == 'average':
            league_stats_df_by_stat.loc[owner, 'average'] = league_stats_df_by_stat.loc[owner, gameweek_range].mean()
            league_stats_df_by_stat = league_stats_df_by_stat.sort_values('average', ascending=False)
        elif total_type == 'total':
            league_stats_df_by_stat.loc[owner, 'totals'] = league_stats_df_by_stat.loc[owner, gameweek_range].sum()
            league_stats_df_by_stat = league_stats_df_by_stat.sort_values('totals', ascending=False)

    if not sort_by_week == 0:
        league_stats_df_by_stat = league_stats_df_by_stat.sort_values(sort_by_week, ascending=False)
        
    return(league_stats_df_by_stat)

In [235]:
df = get_league_stats_df_by_stat(league_stats_df_by_gameweek_with_bench, 'goal difference', 'total')
pd.concat([df,league_stats_df_by_gameweek_with_bench[current_gameweek][['weekly position']]],axis=1)

,1,2,totals,weekly position
George,-10,-8,-18,6
Emma,-11,-10,-21,3
Luke,-7,-21,-28,2
Olivia,-11,-19,-30,5
Claire,-11,-19,-30,1
Matt,-10,-20,-30,4
Fat,-12,-24,-36,7


In [175]:
stats_df = get_league_stats_df_by_gameweek(all_lineups_df, 2, True)
stat_to_sort_by = 'selected %'
stats_df.sort_values(stat_to_sort_by, ascending=False)[[stat_to_sort_by,'total points','league position']]

TypeError: get_league_stats_df_by_gameweek() takes from 1 to 2 positional arguments but 3 were given